In [27]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
import cv2 as cv
import gradio as gr

In [28]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [29]:
# charger le modèle
model = load_model('./xception_hypermodel.h5', custom_objects={"f1_m": f1_m})

In [30]:
races_labels = ['Australian_terrier', 'Chihuahua', 'English_setter', 'French_bulldog',
 'German_shepherd', 'Labrador_retriever', 'Staffordshire_bullterrier',
 'beagle', 'cocker_spaniel,' 'golden_retriever']

In [31]:
# Définir la fonction de prédiction complète
def races_prediction(inp):
    # Convertir en RGB
    img = cv.cvtColor(inp,cv.COLOR_BGR2RGB)
    # Redimensionner l'image
    dim = (299, 299)
    img = cv.resize(img, dim, interpolation=cv.INTER_LINEAR)
    # Equalization
    img_yuv = cv.cvtColor(img,cv.COLOR_BGR2YUV)
    img_yuv[:,:,0] = cv.equalizeHist(img_yuv[:,:,0])
    img_equ = cv.cvtColor(img_yuv, cv.COLOR_YUV2RGB)
    # Appliquer le filtre non-local means sur l'image de test  
    dst_img = cv.fastNlMeansDenoisingColored(
        src=img_equ,
        dst=None,
        h=10,
        hColor=10,
        templateWindowSize=7,
        searchWindowSize=21)

    # Convertir l'image modifiée en array
    img_array = tf.keras.preprocessing.image.img_to_array(dst_img)
    
    # Appliquer le prétraitement Xception
    img_array = img_array.reshape((-1, 299, 299, 3))
    img_array = tf.keras.applications.xception.preprocess_input(img_array)
    
    # Predictions
    prediction = model.predict(img_array).flatten()
    
    # retour de la prédiction
    return {races_labels[i]: float(prediction[i]) for i in range(len(races_labels))}


# Construire l'interface
image = gr.inputs.Image(shape=(299,299))
label = gr.outputs.Label(num_top_classes=3)

gr.Interface(
    fn=races_prediction,
    inputs=image,
    outputs=label,
    capture_session=True
).launch(share=True)

IMPORTANT: You are using gradio version 2.2.15, however version 2.4.2 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7882/

This share link will expire in 72 hours. To get longer links, send an email to: support@gradio.app
Running on public URL: https://47464.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7882/',
 'https://47464.gradio.app')